Objetivo: Cortar y rasterizar el shapefile de manglares de México para entrenar un clasificador de cobertura para imágenes landsat. 

In [51]:
import landsat.search as search
import landsat.downloader as downloader

In [83]:
scene = search.Search().search('031,045', limit = 100, cloud_max = 1, start_date = "2014", end_date = "2015")

In [84]:
scene['results']

[{'cloud': 0.23,
  'date': u'2014-05-10',
  'path': u'031',
  'row': u'045',
  'sat_type': u'L8',
  'sceneID': u'LC80310452014130LGN00',
  'thumbnail': u'http://earthexplorer.usgs.gov/browse/landsat_8/2014/031/045/LC80310452014130LGN00.jpg'}]

In [85]:
download = downloader.Downloader()

In [ ]:
output = download.download([str(scene['results'][0]['sceneID'])], bands = [1])

In [261]:
!landsat process /Users/RaulSierra/landsat/downloads/LC80310452014130LGN00.tar.bz --bands 432156791011

===> Unzipping LC80310452014130LGN00 - It might take some time
     LC80310452014130LGN00 is already unzipped.
===> Image processing started for bands 4-3-2-1-5-6-7-9-10-11
===> Getting boundaries
===> Projecting
     band 4
     band 3
     band 2
     band 1
     band 5
     band 6
     band 7
     band 9
     band 10
     band 11
===> Calculating cloud and snow coverage from QA band
     cloud/snow coverage: 0.77
===> Final Steps
     Color correcting band 4
     Color correcting band 3
     Color correcting band 2
     Color correcting band 1
     Color correcting band 5
     Color correcting band 6
     Color correcting band 7
     Color correcting band 9
     Color correcting band 10
     Color correcting band 11
     Writing to file
===> The output is stored at /Users/RaulSierra/landsat/processed/LC80310452014130LGN00/LC80310452014130LGN00_bands_432156791011.TIF
===> Done!
Time spent : 99.56 seconds


In [262]:
landsat_dir = "/Users/RaulSierra/landsat"
landsat_scene = landsat_dir + "/processed/LC80310452014130LGN00/LC80310452014130LGN00_bands_432156791011.TIF"
train_dir = "Darwin-SFL-101/landsat/train/"

## Crear raster de entrenamiento

In [202]:
!mapshaper Darwin-SFL-101/mapa_manglar/mexoc2010gw/mexoc2010gw.shp -filter-fields Clase -o manglar_clase.shp

Wrote manglar_clase.shp
Wrote manglar_clase.shx
Wrote manglar_clase.dbf
Wrote manglar_clase.prj


Ahora hay que recortar el shape a solo la escena que vamos a usar

In [263]:
!rio bounds $landsat_scene

/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/warp.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._warp import _transform_geom, _reproject, RESAMPLING
/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/fill.py:2: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._fill import _fillnodata
{"bbox": [-107.31526908421758, 20.59854358911711, -105.07951516054297, 22.730735318220372], "features": [{"bbox": [-107.31526908421758, 20.59854358911711, -105.07951516054297, 22.730735318220372], "geometry": {"coordinates": [[[-107.31526908421758, 20.59854358911711], [-105.07951516054297, 20.59854358911711], [-105.07951516054297, 22.730735318220372], [-107.31526908421758, 22.730735318220372], [-107.31526908421758, 20.59854358911711]]], "type": "Polygon"}, "properties": {"filename": "LC80310452014130LGN00_bands_432156791011.TIF", "id": "0", "title": "/Users/Raul

In [264]:
bbox = "-107.31526908421758,20.59854358911711,-105.07951516054297,22.730735318220372"
!mapshaper manglar_clase.shp -clip bbox=$bbox -o manglar_clase_clip.shp 

[filter] Retained 9,518 of 115,302 features
Wrote manglar_clase_clip-ms.shp
Wrote manglar_clase_clip-ms.shx
Wrote manglar_clase_clip-ms.dbf
Wrote manglar_clase_clip-ms.prj


In [265]:
!rio info $landsat_scene

/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/warp.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._warp import _transform_geom, _reproject, RESAMPLING
/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/fill.py:2: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._fill import _fillnodata
{"count": 10, "crs": "EPSG:3857", "photometric": "RGB", "interleave": "pixel", "dtype": "uint8", "driver": "GTiff", "transform": [32.5294717451994, 0.0, -11946281.108798238, 0.0, -32.69938230068562, 2599487.9455454494], "lnglat": [-106.19739212238026, 21.66857998213058], "height": 7811, "width": 7651, "shape": [7811, 7651], "tiled": false, "res": [32.5294717451994, 32.69938230068562], "nodata": 0.0, "bounds": [-11946281.108798238, 2344073.070394794, -11697398.120475717, 2599487.9455454494]}


In [286]:
!gdal_rasterize -h

Missing source or destination.

Usage: gdal_rasterize [-b band]* [-i] [-at]
       [-burn value]* | [-a attribute_name] [-3d]
       [-l layername]* [-where expression] [-sql select_statement]
       [-of format] [-a_srs srs_def] [-co "NAME=VALUE"]*
       [-a_nodata value] [-init value]*
       [-te xmin ymin xmax ymax] [-tr xres yres] [-tap] [-ts width height]
       [-ot {Byte/Int16/UInt16/UInt32/Int32/Float32/Float64/
             CInt16/CInt32/CFloat32/CFloat64}] [-q]
       <src_datasource> <dst_filename>


In [1]:
!gdal_rasterize -a_nodata 0 -a Clase -init 0 -ts 7651 7811 -ot Int16 -l manglar_clase_clip manglar_clase_clip.shp gt_manglar_clase_clip_int.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [2]:
!rio info gt_manglar_clase_clip_int.tif

/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/warp.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._warp import _transform_geom, _reproject, RESAMPLING
/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/fill.py:2: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._fill import _fillnodata
{"count": 1, "crs": "EPSG:4326", "interleave": "band", "dtype": "int16", "driver": "GTiff", "transform": [9.995573828773086e-05, 0.0, -105.92160746356771, 0.0, -0.000271381981135052, 22.730735318220372], "lnglat": [-105.539226786748, 21.670852990897423], "height": 7811, "width": 7651, "shape": [7811, 7651], "tiled": false, "res": [9.995573828773086e-05, 0.000271381981135052], "nodata": 0.0, "bounds": [-105.92160746356771, 20.61097066357448, -105.15684610992828, 22.730735318220372]}


In [222]:
!mapshaper manglar_clase_clip.shp -info

Name: manglar_clase_clip
Geometry: polygon
Records: 9,518
Bounds: -105.92160746356771 20.61097066357448 -105.15684610992828 22.730735318220372
Data table
  Field  First value
  Clase  1


In [ ]:
!rio info 

In [223]:
!ogr2ogr -f GeoJSON -t_srs crs:84 manglar_clase_clip.geojson manglar_clase_clip.shp

In [226]:
base_raster = "/Users/RaulSierra/landsat/processed/LC80310452014130LGN00/LC80310452014130LGN00_bands_564_pan.TIF"
!mapshaper combine-files test_combine1.geojson test_combine2.geojson -o test_merge.geojson

Wrote test_merge1.geojson
Wrote test_merge2.geojson


In [192]:
rapideye_name = "1349112_2014-05-17_RE4_3A_255761"
!rio clip $landsat_scene {"LS_" + rapideye_name  + ".tif"} --like {train_dir + rapideye_name + ".tif"}

/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/warp.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._warp import _transform_geom, _reproject, RESAMPLING
/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/fill.py:2: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._fill import _fillnodata


In [195]:
!rio warp {train_dir + rapideye_name + ".gt"} {train_dir + "landsat/LS_" + rapideye_name  + ".gt"} --like {train_dir + "landsat/LS_" + rapideye_name  + ".tif"}

/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/warp.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._warp import _transform_geom, _reproject, RESAMPLING
/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/fill.py:2: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._fill import _fillnodata


In [197]:
!rio info {train_dir + "landsat/LS_" + rapideye_name  + ".tif"}

/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/warp.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._warp import _transform_geom, _reproject, RESAMPLING
/Users/RaulSierra/.virtualenvs/eda/lib/python2.7/site-packages/rasterio/fill.py:2: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from rasterio._fill import _fillnodata
{"count": 11, "crs": "EPSG:3857", "interleave": "pixel", "dtype": "uint8", "driver": "GTiff", "transform": [32.5294717451994, 0.0, -11766718.424764737, 0.0, -32.69938230068562, 2502436.1788770147], "lnglat": [-105.58066768689272, 21.810871901693353], "height": 832, "width": 832, "shape": [832, 832], "tiled": false, "res": [32.5294717451994, 32.69938230068562], "nodata": 0.0, "bounds": [-11766718.424764737, 2475230.292802844, -11739653.904272731, 2502436.1788770147]}
